<a href="https://colab.research.google.com/github/AlanDSigala/Momentos-de-Hu/blob/main/Proyecto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reconocimiento de formas

In [ ]:
! pip install kaggle

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d smeschke/four-shapes

 83% 18.0M/21.6M [00:00<00:00, 94.8MB/s]
100% 21.6M/21.6M [00:00<00:00, 95.9MB/s]


In [ ]:
! unzip four-shapes.zip

In [ ]:
import os
import cv2
import numpy as np
import random

dataset_path = '/content/shapes'

# Lista para almacenar las rutas de las imágenes
imagenes = []

# Recorre las carpetas y guarda las rutas de las imágenes en la lista
for clase in os.listdir(dataset_path):
    clase_path = os.path.join(dataset_path, clase)
    if os.path.isdir(clase_path):
        for imagen_file in os.listdir(clase_path):
            imagen_path = os.path.join(clase_path, imagen_file)
            imagenes.append((imagen_path, clase))

# Baraja aleatoriamente las imagenes
random.shuffle(imagenes)

# Variables para contar las imagenes de cada clase en la muestra
contador_por_clase = {'circulo': 0, 'cuadrado': 0, 'triangulo': 0, 'star': 0}

# Lista para almacenar las imagenes de la muestra
muestra = []

# Función para calcular momentos invariantes de Hu
def calcular_momentos_invariantes(imagen):
    imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    momentos = cv2.moments(imagen_gris)

    # Calcular momentos invariantes de Hu
    hu_moments = cv2.HuMoments(momentos)

    # Escalar los momentos para hacerlos más robustos a cambios de escala
    hu_moments = -1 * np.sign(hu_moments) * np.log10(np.abs(hu_moments))

    # Devolver los momentos invariantes como un array unidimensional
    return hu_moments.flatten()

# Itera sobre las imagenes y añade a la muestra hasta alcanzar 1000 patrones
for imagen_path, clase in imagenes:
    imagen = cv2.imread(imagen_path)
    momentos_invariantes = calcular_momentos_invariantes(imagen)


    if contador_por_clase.get(clase, 0) < 250:
        muestra.append((momentos_invariantes, clase))
        contador_por_clase[clase] = contador_por_clase.get(clase, 0) + 1

    if sum(contador_por_clase.values()) == 1000:
        break

# Verificar que la muestra tenga 1000 patrones
print("Número de patrones en la muestra:", len(muestra))


Número de patrones en la muestra: 1000


In [ ]:
for i in range(10):
    print(f"Imagen {i + 1}:")
    print("Momentos Invariantes:", muestra[i][0])
    print("Clase:", muestra[i][1])
    print("----")

In [ ]:
# Crear una lista de encabezados para el archivo ARFF
encabezados = []
for i in range(len(muestra[0][0])):
    encabezados.append(f'feature_{i+1}')

# Crear el contenido del archivo ARFF
arff_content = f"@RELATION formas\n\n"

# Añadir encabezados de atributos
for encabezado in encabezados:
    arff_content += f"@ATTRIBUTE {encabezado} NUMERIC\n"

# Añadir la etiqueta de clase
etiquetas_clases = set([clase for _, clase in muestra])
clases_str = "{" + ",".join(etiquetas_clases) + "}"
arff_content += f"@ATTRIBUTE class {clases_str}\n\n"

# Añadir datos
arff_content += "@DATA\n"
for momentos, clase in muestra:
    arff_content += ",".join(map(str, momentos)) + f",{clase}\n"

# Guardar el contenido en un archivo ARFF
with open('/content/drive/MyDrive/shape_forms.arff', 'w') as f:
    f.write(arff_content)